<a href="https://colab.research.google.com/github/vivekvision/PySparkMLRepo/blob/main/PySparkLinearRegEcom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 64.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=cc08f69b79ac293ae9f0033328791dbbccff43b36bbb3340066dea9efaf2d302
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
#spark

In [5]:
from pyspark.ml.regression import LinearRegression


In [6]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.limit(10).toPandas()

,Email,Address,Avatar,Avg Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank TunnelWrightmouth, MI 82180-9605",Violet,34.497268,12.655651,39.577668,4.082621,587.951054
1,hduke@hotmail.com,"4547 Archer CommonDiazchester, CA 06566-8576",DarkGreen,31.926272,11.109461,37.268959,2.664034,392.204933
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582Cobbborough, DC ...",Bisque,33.000915,11.330278,37.110597,4.104543,487.547505
3,riverarebecca@gmail.com,"1414 David ThroughwayPort Jason, OH 22070-1220",SaddleBrown,34.305557,13.717514,36.721283,3.120179,581.852344
4,mstephens@davidson-herman.com,"14023 Rodriguez PassagePort Jacobville, PR 372...",MediumAquaMarine,33.330673,12.795189,37.536653,4.446308,599.406092
5,alvareznancy@lucas.biz,"645 Martha Park Apt. 611Jeffreychester, MN 672...",FloralWhite,33.871038,12.026925,34.476878,5.493507,637.102448
6,katherine20@yahoo.com,"68388 Reyes Lights Suite 692Josephbury, WV 922...",DarkSlateBlue,32.021596,11.366348,36.683776,4.685017,521.572175
7,awatkins@yahoo.com,Unit 6538 Box 8980DPO AP 09026-4941,Aqua,32.739143,12.351959,37.373359,4.434273,549.904146
8,vchurch@walter-martinez.com,"860 Lee KeyWest Debra, SD 97450-0495",Salmon,33.987773,13.386235,37.534497,3.273434,570.200409
9,bonnie69@lin.biz,"PSC 2734, Box 5255APO AA 98456-7482",Brown,31.936549,11.814128,37.145168,3.202806,427.199385


In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
# inputCols is list of columns that will be used in model 
assembler =  VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website',\
                                        'Length of Membership'], outputCol='features')

output = assembler.transform(data)

In [13]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [11]:
final_data = output.select('features', 'Yearly Amount Spent')

In [15]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [12]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                359|
|   mean| 499.19209326061025|
| stddev|   78.2267267122035|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                141|
|   mean|  499.6245223314629|
| stddev|  82.30409138254495|
|    min|  282.4712457199145|
|    max|  744.2218671047146|
+-------+-------------------+



In [13]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [14]:
lr_model = lr.fit(train_data)

In [15]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------+
|          residuals|
+-------------------+
| -6.189977188697469|
|-0.7271073664351775|
| -5.648577298435782|
|  -7.42729010422255|
| -13.96922338814801|
|  9.125204140818141|
| 20.627199856092147|
| 1.8650050876332216|
| 2.2786279995211203|
| -6.263802696246216|
| -9.151981079265113|
|  3.521938668956693|
|-15.462292466723966|
|  17.22995543229399|
|-5.2509413258649715|
|  6.095095207049724|
| -27.27373397949816|
|  -7.34800029294172|
|-2.3984659624968003|
| -18.77487344769912|
+-------------------+
only showing top 20 rows



In [16]:
test_results.rootMeanSquaredError

9.98484829350356

In [17]:
test_results.r2

0.9853430787734897

In [18]:
unlabeled_data = test_data.select('features')

In [19]:
predictions = lr_model.transform(unlabeled_data)

In [20]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|290.01740647278984|
|[30.8364326747734...|473.71570698829237|
|[30.8794843441274...| 495.7010637904084|
|[31.0472221394875...| 389.5105746729282|
|[31.0662181616375...| 463.9033586564094|
|[31.2681042107507...| 429.3058659690005|
|[31.3091926408918...| 431.5636662525653|
|[31.3123495994443...|445.98528633666524|
|[31.5147378578019...| 495.9400053748668|
|[31.5171218025062...|281.33646483941334|
|[31.5261978982398...| 419.0797568677374|
|[31.5316044825729...| 433.3684823633771|
|[31.5741380228732...| 560.1163153561581|
|[31.7207699002873...|  546.946696115943|
|[31.7366356860502...|  496.598045547212|
|[31.8093003166791...|  549.719706322205|
|[31.8124825597242...|398.34654668908934|
|[31.8164283341993...| 519.4967285213138|
|[31.8530748017465...| 462.7438176047715|
|[31.9048571310136...|492.26253558803137|
+--------------------+------------